# Detectron2 Training

## Install Detectron2

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

## Get Dataset

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="89GbLMgCp3O4PZ2Dx9kd")
project = rf.workspace("team0").project("tennis-ball-tracking-mgygr")
version = project.version(6)
dataset = version.download("coco")

In [ ]:
dataPath = dataset.location
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

# 註冊訓練集
register_coco_instances("my_dataset_train", {}, "dataPath/train/_annotations.coco.json", "dataPath/train")

# 註冊驗證集
register_coco_instances("my_dataset_val", {}, "dataPath/val/_annotations.coco.json", "dataPath/val")

In [ ]:
config ='''
ATASETS:
  TRAIN: ("my_dataset_train", )
  TEST: ("my_dataset_val", )
MODEL:
  ROI_HEADS:
    NUM_CLASSES: 1
'''
!touch ./MyConfig.yaml
!echo config > ./MyConfig.yaml

In [ ]:
import random
import cv2
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog

dataset_dicts = DatasetCatalog.get("my_dataset_train")
metadata = MetadataCatalog.get("my_dataset_train")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata)
    out = visualizer.draw_dataset_dict(d)
    cv2.imshow("sample", out.get_image()[:, :, ::-1])
    cv2.waitKey(0)
cv2.destroyAllWindows()

### 

## Training

In [ ]:
!python detectron2/tools/train_net.py --num-gpus 1 --config-file ./MyConfig.yaml